Load file


In [9]:
import datetime
import pandas as pd
import pyodbc

def execute_job():
    data = datetime.datetime.now().date().strftime("%Y%m%d")
    path = rf"Z:\\Rollout\\R_{data}.csv"
    commit = 'replace'  # Option: { replace, insert }
    
    try:
        read_f = pd.read_csv(path)
        valid_f = True
        print('Arquivo lido com sucesso')
    except FileNotFoundError:
        valid_f = False
        print(f'Arquivo não encontrado: {path}')
    
    if valid_f:
        lkd_server = '172.17.1.35'
        db = 'dw'
        usr = 'USR_MIS'
        pwd = 'MIS@321'
        path_db = f'DRIVER={{SQL Server}};SERVER={lkd_server};DATABASE={db};UID={usr};PWD={pwd}'
    
        connection = pyodbc.connect(path_db)
        cursor = connection.cursor()

        print('Processando arquivo')
        cursor.fast_executemany = True

        desired_order = ['idcliente', 'idcaso', 'dsagencia', 'dsranking', 'DescontoAntigo', 'DescontoNovo', 'DESCONTO']
        
        # Verifique as colunas no DataFrame
        print('Colunas no DataFrame:', read_f.columns.tolist())
        read_f = read_f[desired_order]

        # Verifique valores nulos
        print('Valores nulos nas colunas:', read_f.isnull().sum())

        batch_size = 100000
    
        cursor.execute("TRUNCATE TABLE [recovery].[rollout_20240717]")
        connection.commit()
        print('Tabela truncada com sucesso')

        for start in range(0, len(read_f), batch_size):
            end = start + batch_size
            batch = read_f.iloc[start:end]
            
            data_to_insert = [
                (row['idcliente'], row['idcaso'], row['dsagencia'], row['dsranking'], 
                    row['DescontoAntigo'], row['DescontoNovo'], row['DESCONTO'])
                for index, row in batch.iterrows()
            ]

            cursor.executemany(
                """
                INSERT INTO [recovery].[rollout_20240717] 
                (IdCliente, IdCaso, DsAgencia, dsranking, DescontoAntigo, DescontoNovo, DESCONTO, DataImportacao) 
                VALUES (?, ?, ?, ?, ?, ?, ?, GETDATE())
                """,
                data_to_insert
            )

            connection.commit()
            print(f'Lote {start // batch_size + 1} inserido com sucesso')

        print('Rodou replace com sucesso')
       
    
execute_job();

Arquivo lido com sucesso
Processando arquivo
Colunas no DataFrame: ['idcliente', 'idcaso', 'dsagencia', 'dsranking', 'DescontoAntigo', 'DescontoNovo', 'DESCONTO']
Valores nulos nas colunas: idcliente         0
idcaso            0
dsagencia         0
dsranking         0
DescontoAntigo    0
DescontoNovo      0
DESCONTO          0
dtype: int64
Tabela truncada com sucesso
Lote 1 inserido com sucesso
Lote 2 inserido com sucesso
Lote 3 inserido com sucesso
Lote 4 inserido com sucesso
Rodou replace com sucesso
